In [2]:
from imutils import contours
import numpy as np
import argparse
import cv2
import myutils

In [23]:
def order_points(pts):
	# 一共4个坐标点
	rect = np.zeros((4, 2), dtype = "float32")

	# 按顺序找到对应坐标0123分别是 左上，右上，右下，左下
	# 计算左上，右下
	s = pts.sum(axis = 1)
	rect[0] = pts[np.argmin(s)]
	rect[2] = pts[np.argmax(s)]

	# 计算右上和左下
	diff = np.diff(pts, axis = 1)
	rect[1] = pts[np.argmin(diff)]
	rect[3] = pts[np.argmax(diff)]

	return rect

In [24]:
def resize(image, width=None, height=None, inter=cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]
    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))
    resized = cv2.resize(image, dim, interpolation=inter)
    return resized

In [25]:
def four_point_transform(image, pts):
	# 获取输入坐标点
	rect = order_points(pts)
	(tl, tr, br, bl) = rect

	# 计算输入的w和h值
	widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
	widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
	maxWidth = max(int(widthA), int(widthB))

	heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
	heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
	maxHeight = max(int(heightA), int(heightB))

	# 变换后对应坐标位置
	dst = np.array([
		[0, 0],
		[maxWidth - 1, 0],
		[maxWidth - 1, maxHeight - 1],
		[0, maxHeight - 1]], dtype = "float32")

	# 计算变换矩阵
	M = cv2.getPerspectiveTransform(rect, dst)
	warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

	# 返回变换后结果
	return warped

In [4]:
def cv_show(name,img):
    cv2.imshow(name,img)
    cv2.waitKey(0)
    cv2.destroyAllWindows();

In [49]:
receipt = cv2.imread('scan/receipt.jpg')
ratio = receipt.shape[0]/500.0
orig = receipt.copy()
receipt = resize(orig,height=500)
# receipt = cv2.resize(receipt,(200,300))
cv_show('receipt',receipt)

In [50]:
gray_img = cv2.cvtColor(receipt,cv2.COLOR_BGR2GRAY)
gray_img = cv2.GaussianBlur(gray_img,(5,5),0)
edged = cv2.Canny(gray_img,75,200)

contours,hierarchy = cv2.findContours(edged.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
draw_img = receipt.copy()
res = cv2.drawContours(draw_img,contours,-1,(0,0,255),3)
# cv_show('res',draw_img)

In [51]:
cnts = sorted(contours,key = cv2.contourArea,reverse = True)[:5]

In [52]:
# draw_img2 = receipt.copy()
# tmpres = cv2.drawContours(draw_img2,cnts[0],-1,(0,255,0),3)
# cv_show('tmpres',tmpres)

In [53]:

for (i,c) in enumerate(cnts):
    
    peri = cv2.arcLength(c,True);
    
    approx = cv2.approxPolyDP(c,0.02*peri,True);
    
    if len(approx) == 4:
        screencnt = approx;
        break;
        
        
tmp = receipt.copy();
cv2.drawContours(tmp,[screencnt],-1,(0,255,0),3)
cv_show('outlines',tmp)

In [54]:
warped = four_point_transform(orig,screencnt.reshape(4,2)*ratio);

In [55]:
warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
cv_show('warped',warped)

In [56]:
ref = cv2.threshold(warped, 100, 255, cv2.THRESH_BINARY)[1]

In [58]:
cv2.imshow("Original", resize(orig, height = 650))
cv2.imshow("Scanned", resize(ref, height = 650))
cv2.waitKey(0)

113